In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 400
pd.options.display.max_columns = 400
%load_ext autoreload
%autoreload 2
from loadcreon import LoadCreon

In [2]:
path = 'C:\Data\membership14_final.txt'
lc = LoadCreon(path)

## Let's stratify 80/20 train test holdout for future use

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(lc.X, lc.y, test_size=0.2, random_state=771, stratify=lc.y)

## Let's just train on PN data for now

In [8]:
from semisuperhelper import SemiSupervisedHelper
semi_helper = SemiSupervisedHelper(y_train)

In [9]:
X_train_pn, y_train_pn = semi_helper.pn(X_train)

## Quick Scaler and LASSO to test run-time

In [10]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [11]:
estimators = [('scaler', MaxAbsScaler()),('clf',LogisticRegression(penalty='l1'))]
pipe = Pipeline(estimators)

In [19]:
from sklearn.model_selection import cross_val_score, cross_val_predict

In [22]:
%%time
scores = cross_val_score(pipe, X_train_pn, y_train_pn, cv=10, scoring='f1', n_jobs=4)

Wall time: 2.99 s


In [23]:
scores

array([ 0.98787879,  0.98787879,  0.98489426,  0.98480243,  0.98480243,
        0.98480243,  0.9939759 ,  0.97546012,  0.98787879,  0.98787879])

In [25]:
scores.mean(), scores.std()*2

(0.98602527324805533, 0.0088168523106660172)

In [26]:
X_train_u, y_u = semi_helper.u(X_train)

In [27]:
pipe.fit(X_train_pn, y_train_pn)

Pipeline(steps=[('scaler', MaxAbsScaler(copy=True)), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [28]:
y_predict_u = pipe.predict(X_train_u)

In [29]:
pd.Series(y_predict_u).value_counts()

1    349904
0       120
dtype: int64